In [1]:
library(feather)
library(varbvs)

In [2]:
get_rds_toy <- function(datalist){
    for (dat in datalist){
        newdat = read_feather(dat)
        newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
        colnames(newmatrix) = colnames(newdat)
        newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                        logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
        newfit$summary = summary(newfit)
        newfit$w = normalizelogweights(newfit$logw)
        newfit$pip = newfit$alpha %*% c(newfit$w)
        filename = paste(dat, ".RDS", sep="")
        saveRDS(newfit, filename)
        return (newfit$summary$top.vars)
    }
}

In [3]:
get_rds_toy(c("data/toy_n000_600_n100_100_n010_100_n001_100_n110_100_n011_100_n111_100.feather"))

index,variable,prob,PVE,coef,Pr(coef.>0.95)
1,gene1,0.999999816,NA,0.8815534,"[+0.620,+1.157]"
2,gene2,0.012486887,NA,-0.1412246,"[-0.394,+0.098]"
3,gene3,0.009969412,NA,-0.1118736,"[-0.377,+0.150]"


In [4]:
newdat = read_feather("data/toy_n000_600_n100_100_n010_100_n001_100_n110_100_n011_100_n111_100.feather")
newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
colnames(newmatrix) = colnames(newdat)
# print (dim(newdat)[1])
newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
newfit$summary = summary(newfit)
newfit$w = normalizelogweights(newfit$logw)
newfit$pip = newfit$alpha %*% c(newfit$w)
newfit$beta = newfit$mu %*% c(newfit$w)
newfit$var_names = colnames(newdat[-1])
print (summary(newfit))
dimnames(newfit$pip)[[1]]

Summary of fitted Bayesian variable selection model:
family:     binomial   num. hyperparameter settings: 26
samples:    1200       iid variable selection prior: yes
variables:  3          fit prior var. of coefs (sa): no
covariates: 1          fit approx. factors (eta):    yes
maximum log-likelihood lower bound: -817.9226
Hyperparameters: 
        estimate Pr>0.95             candidate values
sa            NA [NA,NA]             1--1
logodds    -1.41 [-2.30,-1.00]       (-3.50)--(-1.00)
Selected variables by probability cutoff:
>0.10 >0.25 >0.50 >0.75 >0.90 >0.95 
    1     1     1     1     1     1 
Top 3 variables by inclusion probability:
  index variable    prob PVE  coef*  Pr(coef.>0.95)
1     1    gene1 1.00000  NA  0.882 [+0.615,+1.143]
2     2    gene2 0.01249  NA -0.141 [-0.373,+0.095]
3     3    gene3 0.00997  NA -0.112 [-0.366,+0.158]
*See help(varbvs) about interpreting coefficients in logistic regression.


[1] "gene1" "gene2" "gene3"